# Mosquitoes detection demo

In [ ]:
# the notebook will reload external python modules;
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.append('../codes')
sys.path.append('../codes/maskrcnn-benchmark/')

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

import requests
from io import BytesIO
from PIL import Image
import numpy as np
import cv2

from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.data.datasets import MosquitoesCOCODataset
from demo.predictor import MosquitoDemo
from utils.img_utils import add_bboxes_on_image

In [ ]:
# this makes our figures bigger
pylab.rcParams['figure.figsize'] = 14, 10

In [ ]:
root = '../data/CEFET/VideoDataSet/5m'
dataset = 'Test'
data = MosquitoesCOCODataset(
    ann_file=os.path.join(root, dataset, 'coco_format_'+dataset+'.json'),    
    root=os.path.join(root, dataset),
    remove_images_without_annotations=True
    )

In [ ]:
exp = 'e2e_faster_rcnn_R_50_C4_1x_cocostyle'
model = 'model_final'

config_file = '../codes/maskrcnn-benchmark/configs/mosquitoes/{}.yaml'.format(exp)
weights = "../codes/maskrcnn-benchmark/results/{}/{}.pth".format(exp, model)

# update the config options with the config file
cfg.merge_from_file(config_file)
# manual override some options
cfg.merge_from_list(["MODEL.DEVICE", "cuda"])
cfg.merge_from_list(["DATALOADER.SIZE_DIVISIBILITY", 32])
cfg.merge_from_list(["MODEL.WEIGHT", weights])

In [ ]:
mosquitoes_demo = MosquitoDemo(
    cfg,
    min_image_size=800,
    confidence_threshold=0.7,
)

In [ ]:
img, target, idx = data[105]
img = np.array(img)
# predictor considers images in BGR channel order
pred = mosquitoes_demo.compute_prediction(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

In [ ]:
result = img.copy()
result = add_bboxes_on_image(result, target.bbox, thickness=3, color=(0, 255, 0))
result = add_bboxes_on_image(result, pred.bbox, thickness=3, color=(0, 0, 255))
result = mosquitoes_demo.overlay_class_names(result, pred)
plt.imshow(result)
plt.axis('off')
plt.show()

## Testing model on arbitrary images

In [ ]:
def load_url(url):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    response = requests.get(url)
    pil_image = Image.open(BytesIO(response.content)).convert("RGB")
    image = np.array(pil_image)
    return image

def disp_bbs(orig_img, predictions, thickness=5, ground_truth=None):
    img_bb = orig_img.copy()
    
    if ground_truth is not None:
        img_bb = add_bboxes_on_image(img_bb, target.bbox, thickness=thickness, color=(0, 255, 0))
    
    img_bb = add_bboxes_on_image(img_bb, predictions.bbox,thickness=thickness, color=(0, 0, 255))
    img_bb = mosquitoes_demo.overlay_class_names(img_bb, predictions)
    plt.imshow(img_bb)
    plt.axis('off')
    plt.show()
    return

In [ ]:
image = load_url("https://isiria.files.wordpress.com/2009/03/oxford_tire_pile_011.jpg")
# compute predictions
# predictor considers images in BGR channel order
predictions = mosquitoes_demo.compute_prediction(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
disp_bbs(image, predictions, thickness=1)

In [ ]:
image = load_url("https://thesocietypages.org/socimages/files/2008/12/oxford_tire_pile_09a.jpg")
# compute predictions
# predictor considers images in BGR channel order
predictions = mosquitoes_demo.compute_prediction(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
disp_bbs(image, predictions, thickness=1)

In [ ]:
image = load_url("https://www.dailyserving.com/wp-content/uploads/2015/07/Fig.-1-Edward-Burtynsky-600x467.jpg")
# compute predictions
# predictor considers images in BGR channel order
predictions = mosquitoes_demo.compute_prediction(cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
disp_bbs(image, predictions, thickness=1)